In [ ]:
# ---------------------- IMPORT THE LIBRARIES ----------------------
from math import *
import numpy as np
from scipy.optimize import fsolve
from scipy import signal
import matplotlib
import matplotlib.pyplot as plt

print("Sucess!")

Sucess!


In [ ]:
# ---------------------- Physical Parameters ----------------------
# Dimensions
b = 500E-3 #[m]
d = 150E-3 #[m]
l_beam = 0.812 #[m] (length of beam A)

h = 500E-3 #[m] (Side length of weld) <-- Adjust this one

# Loadings
F_axial    = 1088.7343E3 #[N]
F_bending  = 1659.6679E3 #[N]

# Material Properties
# Beam properties Hot Rolled
Sy_D6AC = 1430 #[MPa]
Sut_D6AC = 1650 #[MPa]
# Electrode properties () E120XX
Sy_electrode = 737 #[MPa] 
Sut_electrode = 827 #[MPa]


In [ ]:
# ---------------------- Primary, Secondary Shear (bending) ----------------------

# Primary shear is Tau_p = F/A
A = 1.414 * h * (b + d) # Shigley, Pg. 513
Tau_p = F_bending / A
print("Tau_p = %0.4f [MPa]" % (Tau_p/1E6))

# Secondary shear is Tau_pp = Mc/I
# Moment of inertia I = 0.707*h*I_u
I_u = ((d**2)/6) * (3*b + d)
I = 0.707*h*I_u
Tau_pp = ((F_bending*l_beam) * (d/2)) / (I)
print("Tau_pp = %0.4f [MPa]" % (Tau_pp/1E6))

#Tau_p and Tau_pp are perpindicular therefore vector sum them
Tau_bending = sqrt(Tau_p**2+Tau_pp**2)
print("Tau_bending = %0.4f [MPa]" % (Tau_bending/1E6))

Tau_p = 3.6115 [MPa]
Tau_pp = 46.2098 [MPa]
Tau_bending = 46.3507 [MPa]


In [ ]:
#----------------------- Primary Shear (axial)------------------------------------

#Primary Shear is Tau_p_axial = F/A
Tau_p_axial = F_axial / A

#no Secondary Shear therefore Tau_p_axial = Tau_axial
Tau_axial = Tau_p_axial
print("Tau_axial = %0.4f [Mpa]" % (Tau_axial/1E6))

Tau_axial = 2.3691 [Mpa]


In [ ]:
#-----------------------Total Shear----------------------------------------------

#Tau_total is the sum of the bending and axial shear
Tau_total = sqrt(Tau_bending**2+Tau_axial**2)
print("Tau_total = %0.4f [Mpa]" % (Tau_total/1E6))

Tau_total = 46.4112 [Mpa]


In [ ]:
#---------------------Safety Factor----------------------------------------------

#determine the Permissible stress for beam material
Tau_allowable_beam=min(0.3*Sut_D6AC, 0.4*Sy_D6AC)
Tau_allowable_electrode=min(0.3*Sut_electrode, 0.4*Sy_electrode)
print("Tau_allowable_electrode = %0.4f [Mpa] for electrode" % (Tau_allowable_electrode))
Tau_allowable = min(Tau_allowable_beam, Tau_allowable_electrode)

print("Tau_allowable = %0.4f [Mpa] for electrode" % (Tau_allowable))

#safety factor
nf = Tau_allowable/(Tau_total/1E6)
print("Safety factor = %0.4f" % (nf))


Tau_allowable_electrode = 248.1000 [Mpa] for electrode
Tau_allowable = 248.1000 [Mpa] for electrode
Safety factor = 5.3457


In [ ]:
# ---------------------- Fatigue Analysis - Material Properties ----------------------

# Parent Material
Se_P = 0.5*Sut_D6AC # [MPa]
k_a = 272*Sut_D6AC**(-0.995) # <-- As-forged
k_b = 1
k_c = 0.59 # <-- Torsion
k_d = 1
k_e = 1
k_f = 1
Sse_D6AC = Se_P * k_a * k_b * k_c * k_d * k_e * k_f
Ssu_D6AC = 0.67*Sut_D6AC
print("k_a (parent) = %0.4f \t\t[MPa]" % k_a)
print("Sse_D6AC = %0.4f \t\t[MPa]" % Sse_D6AC)
print("Ssu_D6AC = %0.4f \t\t[MPa]" % Ssu_D6AC)

# Electrode Material
Se_P = 0.5*Sut_electrode # [MPa]
k_a = 272*Sut_electrode**(-0.995) # <-- As-forged
k_b = 1
k_c = 0.59 # <-- Torsion
k_d = 1
k_e = 1
k_f = 1
Sse_electrode = Se_P * k_a * k_b * k_c * k_d * k_e * k_f
Ssu_electrode = 0.67*Sut_electrode
print("\nk_a (electrode) = %0.4f \t[MPa]" % k_a)
print("Sse_electrode = %0.4f \t[MPa]" % Sse_electrode)
print("Ssu_electrode = %0.4f \t[MPa]" % Ssu_electrode)


k_a (parent) = 0.1711 		[MPa]
Sse_D6AC = 83.2680 		[MPa]
Ssu_D6AC = 1105.5000 		[MPa]

k_a (electrode) = 0.3401 	[MPa]
Sse_electrode = 82.9810 	[MPa]
Ssu_electrode = 554.0900 	[MPa]


In [ ]:
# ---------------------- Fatigue Analysis - Alternating and Midrange Stresses ----------------------

# ----- Primary Shear -----
F_max = F_bending + F_axial
F_min = F_bending

Fa = (F_max - F_min) / 2
Fm = (F_max + F_min) / 2

Tau_p_a = Fa/A
Tau_p_m = Fm/A


# ----- Secondary Shear -----
M_max = F_bending*l_beam
M_min = F_bending*l_beam

M_a = (M_max-M_min)/2
M_m = (M_max + M_min)/2

Tau_pp_a = ((M_a) * (d/2)) / (I)
Tau_pp_m = ((M_m) * (d/2)) / (I)

# ----- Total Shear -----
Tau_a = sqrt(Tau_p_a**2+Tau_pp_a**2)
Tau_m = sqrt(Tau_p_m**2 + Tau_pp_m**2)

# Print all the values
print("Fa = %0.4f \t[MN]" % (Fa/1E6))
print("Fm = %0.4f \t[MN]" % (Fm/1E6))
print("M_a = %0.4f \t[MNm]" % (M_a/1E6))
print("M_m = %0.4f \t[MNm]" % (M_m/1E6))
print("Tau_p_a = %0.4f \t[MPa]" % (Tau_p_a/1E6))
print("Tau_p_m = %0.4f \t[MPa]" % (Tau_p_m/1E6))
print("Tau_pp_a = %0.4f \t[MPa]" % (Tau_pp_a/1E6))
print("Tau_pp_m = %0.4f \t[MPa]" % (Tau_pp_m/1E6))
print("Tau_a = %0.4f \t\t[MPa]" % (Tau_a/1E6))
print("Tau_m = %0.4f \t[MPa]" % (Tau_m/1E6))

Fa = 0.5444 	[MN]
Fm = 2.2040 	[MN]
M_a = 0.0000 	[MNm]
M_m = 1.3477 	[MNm]
Tau_p_a = 1.1846 	[MPa]
Tau_p_m = 4.7961 	[MPa]
Tau_pp_a = 0.0000 	[MPa]
Tau_pp_m = 46.2098 	[MPa]
Tau_a = 1.1846 		[MPa]
Tau_m = 46.4580 	[MPa]


In [ ]:
#----------------------Safety Factor: Goodman Criteria--------------------
Sse_min = min(Sse_D6AC,Sse_electrode)
Ssu_min = min(Ssu_D6AC,Ssu_electrode)

Kfs = 1.5

nf = 1/(((Kfs*Tau_a/1E6)/Sse_min)+((Kfs*Tau_m/1E6)/Ssu_min))

print("nf = %0.4f" % (nf))

nf = 6.7944
